In [ ]:
import pandas as pd
import yfinance as yf  # Assume installed or use pandas_datareader
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import os
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

In [3]:
def fetch_OHLCV(tickers, start_date, end_date):
  # Empty list to store the
  data_ls = []

  # Looping over the required tickers
  for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date).droplevel(axis=1,level=1).reset_index()
    data.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
    data['Ticker'] = ticker
    data_ls.append(data)

  # Creation of the larger dataset
  data_final = pd.concat(data_ls)
  return data_final

In [19]:
# Fetch data function calling
tickers = ['TITAN.NS', 'HDFCBANK.NS', 'ITC.NS', 'SUNPHARMA.NS', 'TECHM.NS']
start='2000-01-01'
end='2025-11-26'

In [37]:
benchmark_symbol = '^NSEI'

df_benchmark = fetch_OHLCV([benchmark_symbol], start, end)
df_benchmark['Returns'] = df_benchmark['Close'].pct_change()
df_benchmark.dropna(inplace=True)

[*********************100%***********************]  1 of 1 completed


In [38]:
def inspect_data(df, viz_dir='preprocessing_viz'):
    """Step 1: Inspect the Data - Summary stats and initial visualizations."""
    os.makedirs(viz_dir, exist_ok=True)
    
    # Print summary stats
    print(f"Shape: {df.shape}")
    print(df.info())
    print(df.describe())
    
    # Viz 1: Timeseries plot of Close prices by Ticker
    fig, ax = plt.subplots(figsize=(12, 6))
    for ticker in df['Ticker'].unique():
        ticker_df = df[df['Ticker'] == ticker]
        ax.plot(ticker_df['Date'], ticker_df['Returns'], label=ticker)
    ax.set_xlabel('Date')
    ax.set_ylabel('Returns')
    ax.set_title('Timeseries of Returns by Ticker')
    ax.legend()
    timeseries_plot = os.path.join(viz_dir, 'timeseries_Returns.png')
    plt.savefig(timeseries_plot)
    plt.close(fig)
    
    # Viz 2: Histogram of Close prices by Ticker
    plt.figure(figsize=(10, 6))
    sns.histplot(data=df, x='Close', hue='Ticker', element='step', bins=20)
    plt.title('Distribution of Close Prices (Inspection)')
    plt.xlabel('Close Price')
    plt.ylabel('Frequency')
    inspect_hist = os.path.join(viz_dir, 'inspect_close_hist.png')
    plt.savefig(inspect_hist)
    plt.close()
    
    return df, [timeseries_plot, inspect_hist]

def handle_missing(df, viz_dir='preprocessing_viz'):
    """Step 2: Handle Missing Values - Visualize and forward-fill."""
    print("Missing before:", df.isnull().sum())
    
    # Viz: Missing matrix
    plt.figure(figsize=(12, 8))
    msno.matrix(df, color=(0.25, 0.5, 0.75))
    plt.title('Missing Value Matrix (Before)')
    missing_matrix = os.path.join(viz_dir, 'missing_matrix_before.png')
    plt.savefig(missing_matrix)
    plt.close()
    
    # Process: Forward-fill per ticker
    df = df.sort_values(['Ticker', 'Date']).copy()
    df[['Open', 'High', 'Low', 'Close', 'Volume']] = df.groupby('Ticker')[['Open', 'High', 'Low', 'Close', 'Volume']].ffill()
    df = df.dropna()
    
    print("Missing after:", df.isnull().sum())
    
    # Viz: Missing bar after
    plt.figure(figsize=(10, 6))
    msno.bar(df, color='skyblue')
    plt.title('Missing Values Bar (After)')
    missing_bar = os.path.join(viz_dir, 'missing_bar_after.png')
    plt.savefig(missing_bar)
    plt.close()
    
    return df, [missing_matrix, missing_bar]

def remove_duplicates(df, viz_dir='preprocessing_viz'):
    """Step 3: Remove Duplicates - Count and drop."""
    dups_before = df.duplicated(subset=['Date', 'Ticker']).sum()
    print(f"Duplicates before: {dups_before}")
    
    # Viz: Bar for duplicates
    plt.figure(figsize=(6, 4))
    plt.bar(['Duplicates'], [dups_before])
    plt.title('Duplicate Count (Before Removal)')
    dups_bar = os.path.join(viz_dir, 'duplicates_bar.png')
    plt.savefig(dups_bar)
    plt.close()
    
    df = df.drop_duplicates(subset=['Date', 'Ticker'], keep='first')
    print(f"Duplicates after: {df.duplicated(subset=['Date', 'Ticker']).sum()}")
    
    return df, [dups_bar]



# Main chaining function (optional: to run all steps sequentially)
def run_preprocessing_pipeline(tickers, start_date, end_date, viz_dir='preprocessing_viz'):
    """Chains all separate functions; returns full results."""
    os.makedirs(viz_dir, exist_ok=True)
    df = fetch_OHLCV(tickers, start_date, end_date)
    df['Returns'] = df.groupby('Ticker')['Close'].pct_change()
    df.dropna(inplace=True)
    all_viz_files = []
    
    # Run each step
    df, viz1 = inspect_data(df, viz_dir)
    all_viz_files.extend(viz1)
    
    df, viz2 = handle_missing(df, viz_dir)
    all_viz_files.extend(viz2)
    
    full_df, viz3 = remove_duplicates(df, viz_dir)
    all_viz_files.extend(viz3)
    

    
    return {
        'preprocessed_df': full_df,
        'all_viz_files': [f for f in all_viz_files if os.path.exists(f)]
    }

# Usage Example:
results = run_preprocessing_pipeline(tickers, start, end)
ohlcv_data = results['preprocessed_df']


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Shape: (30596, 8)
<class 'pandas.core.frame.DataFrame'>
Index: 30596 entries, 1 to 4747
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     30596 non-null  datetime64[ns]
 1   Close    30596 non-null  float64       
 2   High     30596 non-null  float64       
 3   Low      30596 non-null  float64       
 4   Open     30596 non-null  float64       
 5   Volume   30596 non-null  int64         
 6   Ticker   30596 non-null  object        
 7   Returns  30596 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 2.1+ MB
None
                                Date         Close          High  \
count                          30596  30596.000000  30596.000000   
mean   2013-05-16 21:30:05.883121920    417.779407    422.797880   
min              2000-01-04 00:00:00      1.232854      1.291134   
25%              2007-04-19 00:00:00     42.780426     43.848815   
50% 

<Figure size 1200x800 with 0 Axes>

### Performance Metrics

In [39]:
def calculate_performance_ratios(df, df_benchmark = df_benchmark, risk_free_rate=0.00):
    """Calculate performance ratios for each ticker."""

    results = {}

    # Benchmark stats
    benchmark_returns = df_benchmark['Returns']
    benchmark_mean = benchmark_returns.mean()
    benchmark_std = benchmark_returns.std()
    downside_benchmark_std = np.sqrt(np.mean(np.minimum(0, benchmark_returns)**2))
    
    for ticker in df['Ticker'].unique():
        ticker_df = df[df['Ticker'] == ticker]
        returns = ticker_df['Returns']
        
        mean_return = returns.mean()
        std_return = returns.std()
        downside_std = np.sqrt(np.mean(np.minimum(0, returns)**2))
        
      # Sharpe Ratio
        sharpe_ratio = (mean_return - risk_free_rate/252) / std_return
        sharpe_ratio_benchmark = (mean_return - benchmark_mean) / benchmark_std

      # Sortino Ratio
        sortino_ratio = (mean_return - risk_free_rate/252) / downside_std
        sortino_ratio_benchmark = (mean_return - benchmark_mean) / downside_benchmark_std

      # Information Ratio
        information_ratio = (mean_return - benchmark_mean) / np.std(returns - benchmark_returns)
        
        results[ticker] = {
            'Sharpe Ratio': sharpe_ratio,
            'Sharpe Ratio (Benchmark)': sharpe_ratio_benchmark,
            'Sortino Ratio': sortino_ratio,
            'Sortino Ratio (Benchmark)': sortino_ratio_benchmark,
            'Information Ratio': information_ratio
        }
    
    return pd.DataFrame(results).T

In [40]:
performance_ratios_df = calculate_performance_ratios(ohlcv_data)


### Drawdown Metrics

In [41]:
def drawdowns(df, df_benchmark = df_benchmark):
          
    """Calculate maximum drawdowns for each ticker and benchmark."""    
    
    symbols = []
    output = []

    def max_drawdown(returns):
        cumulative = (1 + returns).cumprod()
        peak = cumulative.cummax()
        drawdown = (cumulative - peak) / peak
        return drawdown.min()
    
    # Benchmark drawdown
    benchmark_dd = max_drawdown(df_benchmark['Returns'])
    symbols.append(benchmark_symbol)
    output.append(benchmark_dd)
    # results[benchmark_symbol] = benchmark_dd
    
    for ticker in df['Ticker'].unique():
        ticker_df = df[df['Ticker'] == ticker]
        returns = ticker_df['Returns']
        dd = max_drawdown(returns)
        symbols.append(ticker)
        output.append(dd)
    
    results = pd.DataFrame({'Symbol': symbols, 'Max Drawdown': output},columns=['Symbol', 'Max Drawdown'])
    return results

In [43]:
drawdowns_df = drawdowns(ohlcv_data)

### Returns

In [ ]:
def returns_output(df, df_benchmark, benchmark_symbol='^NSEI'):
          
    """Calculate returns for each ticker and benchmark."""
    
    results = {}

    def calculate_cagr(returns_series, periods_per_year):
        if len(returns_series) == 0:
            return np.nan
        cumulative_return = (1 + returns_series).prod() - 1
        n_years = len(returns_series) / periods_per_year
        if n_years == 0:
            return np.nan
        return (1 + cumulative_return) ** (1 / n_years) - 1

    # Benchmark
    benchmark_returns_full = df_benchmark['Returns']
    results[benchmark_symbol] = {}
    
    results[benchmark_symbol]['1D Return'] = benchmark_returns_full.iloc[-1]
    results[benchmark_symbol]['5D Return'] = benchmark_returns_full.iloc[-5:].sum()
    results[benchmark_symbol]['1M Return'] = benchmark_returns_full.iloc[-21:].sum()
    results[benchmark_symbol]['3M Return'] = benchmark_returns_full.iloc[-63:].sum()
    results[benchmark_symbol]['6M Return'] = benchmark_returns_full.iloc[-126:].sum()
    
    results[benchmark_symbol]['1Y CAGR'] = calculate_cagr(benchmark_returns_full.iloc[-252:], 252)
    results[benchmark_symbol]['3Y CAGR'] = calculate_cagr(benchmark_returns_full.iloc[-756:], 252)
    results[benchmark_symbol]['5Y CAGR'] = calculate_cagr(benchmark_returns_full.iloc[-1260:], 252)
    
    for ticker in df['Ticker'].unique():
        ticker_df = df[df['Ticker'] == ticker].sort_values('Date') if 'Date' in df.columns else df[df['Ticker'] == ticker]
        returns_full = ticker_df['Returns']
        
        results[ticker] = {}
        results[ticker]['1D Return'] = returns_full.iloc[-1]
        results[ticker]['5D Return'] = returns_full.iloc[-5:].sum()
        results[ticker]['1M Return'] = returns_full.iloc[-21:].sum()
        results[ticker]['3M Return'] = returns_full.iloc[-63:].sum()
        results[ticker]['6M Return'] = returns_full.iloc[-126:].sum()
        
        results[ticker]['1Y CAGR'] = calculate_cagr(returns_full.iloc[-252:], 252)
        results[ticker]['3Y CAGR'] = calculate_cagr(returns_full.iloc[-756:], 252)
        results[ticker]['5Y CAGR'] = calculate_cagr(returns_full.iloc[-1260:], 252)
    
    return pd.DataFrame(results).T.round(4)

In [52]:
returns_df = returns_output(ohlcv_data, df_benchmark, benchmark_symbol) 

In [53]:
returns_df

,1D Return,5D Return,1M Return,3M Return,6M Return,1Y CAGR,3Y CAGR,5Y CAGR
^NSEI,-0.0029,-0.0009,0.0037,0.0407,0.0365,0.1006,0.1270,0.1682
HDFCBANK.NS,-0.0094,-0.0025,-0.0128,0.0093,0.0442,0.1523,0.1092,0.1113
ITC.NS,-0.0068,-0.0124,-0.0478,0.0057,-0.0575,-0.0930,0.0790,0.2377
SUNPHARMA.NS,-0.0022,0.0103,0.0486,0.0731,0.0637,0.0075,0.2055,0.3097
TECHM.NS,0.0000,0.0503,0.0233,-0.0060,-0.0246,-0.0965,0.1657,0.1717
TITAN.NS,-0.0050,-0.0061,0.0311,0.0581,0.0831,0.2165,0.1209,0.2607


In [49]:
# Volatility and Tracking
# ● Standard Deviation (Portfolio)
# ● Standard Deviation (Benchmark)
# ● Tracking Error
# ● Rolling Standard Deviation

### Volatility and Tracking

In [50]:
def volatility_tracking(df, df_benchmark = df_benchmark, window=21):
    
    """Calculate volatility and tracking error for each ticker and benchmark."""
    
    results = {}

    # Benchmark stats
    benchmark_returns = df_benchmark['Returns']
    benchmark_std = benchmark_returns.std()
    rolling_benchmark_std = benchmark_returns.rolling(window=window).std().iloc[-1]
    
    results[benchmark_symbol] = {
        'Standard Deviation (Benchmark)': benchmark_std,
        'Rolling Standard Deviation (Benchmark)': rolling_benchmark_std
    }
    
    for ticker in df['Ticker'].unique():
        ticker_df = df[df['Ticker'] == ticker]
        returns = ticker_df['Returns']
        
        std_return = returns.std()
        rolling_std = returns.rolling(window=window).std().iloc[-1]
        
        # Tracking Error
        tracking_error = np.std(returns - benchmark_returns)
        
        results[ticker] = {
            'Standard Deviation (Portfolio)': std_return,
            'Rolling Standard Deviation (Portfolio)': rolling_std,
            'Tracking Error': tracking_error
        }
    
    return pd.DataFrame(results).T

In [54]:
volatility_tracking(ohlcv_data)

,Standard Deviation (Benchmark),Rolling Standard Deviation (Benchmark),Standard Deviation (Portfolio),Rolling Standard Deviation (Portfolio),Tracking Error
^NSEI,0.013079,0.004613,NaN,NaN,NaN
HDFCBANK.NS,NaN,NaN,0.019055,0.006607,0.024575
ITC.NS,NaN,NaN,0.018429,0.006597,0.023767
SUNPHARMA.NS,NaN,NaN,0.021038,0.007440,0.026058
TECHM.NS,NaN,NaN,0.023262,0.013190,0.026917
TITAN.NS,NaN,NaN,0.026811,0.008464,0.032735
